# Random Forests

In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

df = pd.read_csv('../normalizacao/survey_results_salary_numeric.csv', sep=',', encoding='utf-8', index_col=0)
df.head()

,WageAboveAverage,YearsCodeNumber,YearsCodeProNumber,EdLevel_associate,EdLevel_bachelor,EdLevel_doctor,EdLevel_high_school,EdLevel_master,EdLevel_primary_school,EdLevel_professional,EdLevel_unfinished_college,Country_australia,Country_brazil,Country_canada,Country_france,Country_germany,Country_india,Country_italy,Country_netherlands,Country_poland,Country_spain,Country_united_kingdom_of_great_britain_and_northern_ireland,Country_united_states_of_america,Currency_AED,Currency_AUD,Currency_AZN,Currency_BDT,Currency_BRL,Currency_CAD,Currency_CHF,Currency_CZK,Currency_DJF,Currency_EUR,Currency_GBP,Currency_GIP,Currency_IDR,Currency_IMP,Currency_INR,Currency_IRR,Currency_JPY,Currency_KYD,Currency_MAD,Currency_NOK,Currency_NZD,Currency_PLN,Currency_SAR,Currency_TND,Currency_UGX,Currency_USD,Currency_XPF
0,0,14.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,6.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,12.0,10.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,12.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Conversão das variáveis categóricas para numéricas

No arquivo `normalizacao/conversao_categorias.ipynb`.

## Atributos Previsores e Alvo

In [4]:
previsores = df.iloc[:, 0:11].values
previsores

array([[ 0., 14.,  5., ...,  0.,  0.,  0.],
       [ 0.,  6.,  6., ...,  0.,  0.,  0.],
       [ 1.,  5.,  2., ...,  0.,  0.,  0.],
       ...,
       [ 1.,  5.,  1., ...,  0.,  0.,  0.],
       [ 0.,  6.,  2., ...,  0.,  0.,  0.],
       [ 1., 10.,  3., ...,  0.,  0.,  0.]])

In [5]:
previsores.shape

(23265, 11)

In [6]:
alvo = df.iloc[:, 11].values
alvo

array([0., 0., 0., ..., 0., 0., 0.])

In [7]:
alvo.shape

(23265,)

## Análise das escalas dos atributos (Escalonamento)

In [8]:
df.describe()

,WageAboveAverage,YearsCodeNumber,YearsCodeProNumber,EdLevel_associate,EdLevel_bachelor,EdLevel_doctor,EdLevel_high_school,EdLevel_master,EdLevel_primary_school,EdLevel_professional,EdLevel_unfinished_college,Country_australia,Country_brazil,Country_canada,Country_france,Country_germany,Country_india,Country_italy,Country_netherlands,Country_poland,Country_spain,Country_united_kingdom_of_great_britain_and_northern_ireland,Country_united_states_of_america,Currency_AED,Currency_AUD,Currency_AZN,Currency_BDT,Currency_BRL,Currency_CAD,Currency_CHF,Currency_CZK,Currency_DJF,Currency_EUR,Currency_GBP,Currency_GIP,Currency_IDR,Currency_IMP,Currency_INR,Currency_IRR,Currency_JPY,Currency_KYD,Currency_MAD,Currency_NOK,Currency_NZD,Currency_PLN,Currency_SAR,Currency_TND,Currency_UGX,Currency_USD,Currency_XPF
count,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000,23265.000000
mean,0.668687,15.405803,10.531485,0.036364,0.483344,0.041694,0.043542,0.264990,0.003611,0.013540,0.112916,0.032925,0.052181,0.058027,0.056050,0.117301,0.085837,0.032925,0.036321,0.039458,0.040017,0.106168,0.342790,0.000043,0.032839,0.000043,0.000043,0.047840,0.057167,0.000559,0.000043,0.000043,0.283344,0.105996,0.000043,0.000043,0.000043,0.083516,0.000086,0.000043,0.000043,0.000043,0.000043,0.000086,0.035805,0.000043,0.000043,0.000086,0.351988,0.000086
std,0.470696,9.768934,8.545310,0.187197,0.499733,0.199892,0.204078,0.441338,0.059981,0.115572,0.316497,0.178444,0.222397,0.233800,0.230023,0.321785,0.280129,0.178444,0.187091,0.194687,0.196004,0.308059,0.474652,0.006556,0.178219,0.006556,0.006556,0.213432,0.232167,0.023632,0.006556,0.006556,0.450632,0.307839,0.006556,0.006556,0.006556,0.276667,0.009272,0.006556,0.006556,0.006556,0.006556,0.009272,0.185807,0.006556,0.006556,0.009272,0.477601,0.009272
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,8.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,13.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,20.000000,15.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [9]:
from sklearn.preprocessing import StandardScaler

previsores_esc = StandardScaler().fit_transform(previsores)
previsores_esc

array([[-1.42066628, -0.14390853, -0.64732633, ..., -0.06019682,
        -0.11715584, -0.35677649],
       [-1.42066628, -0.96284862, -0.53030055, ..., -0.06019682,
        -0.11715584, -0.35677649],
       [ 0.70389508, -1.06521613, -0.99840366, ..., -0.06019682,
        -0.11715584, -0.35677649],
       ...,
       [ 0.70389508, -1.06521613, -1.11542943, ..., -0.06019682,
        -0.11715584, -0.35677649],
       [-1.42066628, -0.96284862, -0.99840366, ..., -0.06019682,
        -0.11715584, -0.35677649],
       [ 0.70389508, -0.55337857, -0.88137788, ..., -0.06019682,
        -0.11715584, -0.35677649]])

In [10]:
previsoresdf = pd.DataFrame(previsores_esc)
previsoresdf

,0,1,2,3,4,5,6,7,8,9,10
0,-1.420666,-0.143909,-0.647326,-0.194257,-0.967225,-0.208585,-0.213364,1.665450,-0.060197,-0.117156,-0.356776
1,-1.420666,-0.962849,-0.530301,-0.194257,-0.967225,-0.208585,-0.213364,1.665450,-0.060197,-0.117156,-0.356776
2,0.703895,-1.065216,-0.998404,-0.194257,1.033886,-0.208585,-0.213364,-0.600438,-0.060197,-0.117156,-0.356776
3,0.703895,-0.348644,-0.062197,-0.194257,1.033886,-0.208585,-0.213364,-0.600438,-0.060197,-0.117156,-0.356776
4,0.703895,-0.348644,-0.647326,-0.194257,1.033886,-0.208585,-0.213364,-0.600438,-0.060197,-0.117156,-0.356776
...,...,...,...,...,...,...,...,...,...,...,...
23260,-1.420666,-0.860481,-0.998404,5.147815,-0.967225,-0.208585,-0.213364,-0.600438,-0.060197,-0.117156,-0.356776
23261,0.703895,0.572664,0.639957,-0.194257,1.033886,-0.208585,-0.213364,-0.600438,-0.060197,-0.117156,-0.356776
23262,0.703895,-1.065216,-1.115429,-0.194257,1.033886,-0.208585,-0.213364,-0.600438,-0.060197,-0.117156,-0.356776
23263,-1.420666,-0.962849,-0.998404,-0.194257,1.033886,-0.208585,-0.213364,-0.600438,-0.060197,-0.117156,-0.356776


## Base de Treino e Teste

In [11]:
from sklearn.model_selection import train_test_split

x_treino, x_teste, y_treino, y_teste = train_test_split(previsores_esc, alvo, test_size = 0.3, random_state = 0)

In [12]:
x_treino.shape

(16285, 11)

In [13]:
x_teste.shape

(6980, 11)

In [14]:
y_treino.shape

(16285,)

In [15]:
y_teste.shape

(6980,)

## Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier

random = RandomForestClassifier(n_estimators=150, criterion='entropy', random_state = 0, max_depth=4)
random.fit(x_treino, y_treino)

RandomForestClassifier(criterion='entropy', max_depth=4, n_estimators=150,
                       random_state=0)

In [17]:
previsoes_random = random.predict(x_teste)
previsoes_random

array([0., 0., 0., ..., 0., 0., 0.])

In [18]:
y_teste

array([0., 0., 0., ..., 0., 0., 0.])

In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [20]:
print("Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_random) * 100.0))

Acurácia: 96.72%


In [21]:
confusion_matrix(y_teste, previsoes_random)

array([[6751,    0],
       [ 229,    0]], dtype=int64)

In [22]:
print(classification_report(y_teste, previsoes_random))

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      6751
         1.0       0.00      0.00      0.00       229

    accuracy                           0.97      6980
   macro avg       0.48      0.50      0.49      6980
weighted avg       0.94      0.97      0.95      6980



In [23]:
previsoes_treino = random.predict(x_treino)
previsoes_treino

array([0., 0., 0., ..., 0., 0., 0.])

In [24]:
print("Acurácia: %.2f%%" % (accuracy_score(y_treino, previsoes_treino) * 100.0))

Acurácia: 96.70%


In [25]:
confusion_matrix(y_treino, previsoes_treino)

array([[15748,     0],
       [  537,     0]], dtype=int64)